In [116]:
import requests
import json
import pandas as pd

def get_imf_indicators(type_):
    response = requests.get(url="https://www.imf.org/external/datamapper/api/v1/indicators")
    response_body = json.loads(response.text)
    indicators = [{"id": key, **values} for key, values in response_body["indicators"].items()]
    indicators_df = pd.DataFrame.from_records(indicators)
    indicators_df=indicators_df[indicators_df["source"].str.contains(type_, na=False)]
    return indicators_df


def get_imf_groups(group_type):
    response = requests.get(url=f"https://www.imf.org/external/datamapper/api/v1/{group_type}")
    response_body = json.loads(response.text)
    results = [{"id": key, **values} for key, values in response_body[group_type].items()]
    df = pd.DataFrame.from_records(results)
    return df


def get_imf_indicator_data(indicator_id, group_ids, years=None):
    indicator_url = "https://www.imf.org/external/datamapper/api/v1"
    groups_url_path = "/".join(group_ids)
    years_query_param = f"?periods={','.join(years)}" if years else ""
    response = requests.get(url=f"{indicator_url}/{indicator_id}/{groups_url_path}{years_query_param}")
    if response.status_code != 200:
        return pd.DataFrame()
    response_body = response.json()
    response_values = response_body.get("values")
    if not response_values or indicator_id not in response_values:
        return pd.DataFrame()
    all_country_data = response_values[indicator_id]
    indicator_df = pd.DataFrame.from_dict(all_country_data, orient="index").sort_index()
    indicator_df.index.name = "Country"
    return indicator_df

weo=get_imf_indicators("World Economic Outlook")


turkey_indicators_df = pd.DataFrame()

for i, label in zip(weo["id"], weo["label"]):
    try:
        weo_r = get_imf_indicator_data(i, group_ids=["WEO"])
        if "TUR" in weo_r.index:
            turkey_data = weo_r.loc["TUR"]
            turkey_data.name = label
            turkey_indicators_df[label] = turkey_data
        else:
            print(f"TUR not found in {i}")
    except Exception as e:
        print(f"Error processing {i} - {label}: {e}")

turkey_indicators_df.index.name = "Year"
turkey_indicators_df = turkey_indicators_df.sort_index()
turkey_indicators_df


,Real GDP growth,"GDP, current prices","GDP per capita, current prices\n","GDP per capita, current prices","GDP based on PPP, share of world",Implied PPP conversion rate,"Inflation rate, average consumer prices","Inflation rate, end of period consumer prices",Population,Current account balance\nU.S. dollars,"Current account balance, percent of GDP",Unemployment rate,General government net lending/borrowing,General government gross debt
Year,,,,,,,,,,,,,,
1980,-0.8,161.322,2148.973,3588.939,1.222,0.000,110.6,NaN,44.950,-3.090,-3.2,7.2,NaN,NaN
1981,4.4,184.293,2128.209,4007.704,1.246,0.000,36.4,NaN,45.985,-1.901,-1.9,7.2,NaN,NaN
1982,3.4,202.391,1890.083,4302.109,1.283,0.000,31.1,NaN,47.045,-0.952,-1.1,7.6,NaN,NaN
1983,4.8,220.323,1765.895,4579.003,1.310,0.000,31.3,NaN,48.116,-1.923,-2.3,7.5,NaN,NaN
1984,6.8,243.849,1680.418,4958.335,1.340,0.000,48.4,NaN,49.180,-1.439,-1.7,7.4,NaN,NaN
1985,4.3,262.273,1848.330,5222.266,1.348,0.000,44.5,NaN,50.222,-1.013,-1.1,6.9,NaN,NaN
1986,6.9,286.125,1997.258,5584.159,1.396,0.000,34.6,30.7,51.239,-1.465,-1.4,7.7,NaN,NaN
1987,10.0,322.616,2276.858,6176.532,1.479,0.000,38.9,55.0,52.233,-0.806,-0.7,8.1,NaN,NaN
1988,2.1,341.078,2349.943,6410.462,1.447,0.001,73.7,66.2,53.206,1.596,1.3,8.7,NaN,NaN
